<a href="https://colab.research.google.com/github/neha23nagaraj/Movie-Audience-Score-Prediction/blob/main/Part1_WM_FP_SEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 21.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import time
import pandas as pd

def scrape_movie_data_selenium(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)  # Wait for JS content

    movie = {}

    # Title
    try:
        title_element = driver.find_element(By.CSS_SELECTOR, 'rt-text[slot="title"]')
        movie['Title'] = title_element.text.strip()
    except:
        movie['Title'] = None

    # Rating, Release Date, Duration
    try:
      meta_props = driver.find_elements(By.CSS_SELECTOR, 'rt-text[slot="metadataProp"]')

      # Extract their text content by order
      if len(meta_props) >= 3:
         movie['Rating'] = meta_props[0].text.strip().replace(",", "").strip()
         movie['Release Date'] = meta_props[1].text.strip().replace("Released", "").strip().rstrip(',')
         movie['Duration'] = meta_props[2].text.strip().replace(",", "").strip()
      else:
         movie['Rating'] = movie['Release Date'] = movie['Duration'] = None
    except:
        movie['Rating'] = movie['Release Date'] = movie['Duration'] = None

    # Extract Directors (those with "Director" in aria-label)
    try:
        director_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-qa="person-item"] div[slot="insetText"]')
        directors = [el.get_attribute("aria-label").split(",")[0]
                 for el in director_elements
                 if "Director" in el.get_attribute("aria-label")]
        movie['Director'] = ", ".join(directors)
    except:
        movie['Director'] = None

    # Cast (inside shadow DOM-ish structure)
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        cast_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-qa="person-item"] div[slot="insetText"]')

        cast = [el.get_attribute("aria-label").split(",")[0]
            for el in cast_elements
            if "Director" not in el.get_attribute("aria-label")]

        movie['Cast'] = ", ".join(cast)
    except:
        movie['Cast'] = None

    # Synopsis
    try:
        synopsis_element = driver.find_element(By.CSS_SELECTOR, 'rt-text[slot="content"]')
        movie['Synopsis'] = synopsis_element.text.strip()
    except:
        movie['Synopsis'] = None

    # --- Extract Genre ---
    try:
        genre_elements = driver.find_elements(By.CSS_SELECTOR, 'rt-text[slot="metadataGenre"]')
        genres = [g.text.strip() for g in genre_elements]
        movie['Genre'] = ", ".join(genres)  # Join multiple genres into one string
    except:
        movie['Genre'] = None

    # Critic Score (as %)
    try:
        cs = driver.find_element(By.CSS_SELECTOR, 'rt-text[slot="criticsScore"]').text.strip()
        movie['Critic_Score'] = float(cs.replace('%', '')) if cs else None
    except:
        movie['Critic_Score'] = None

    # Critic Review Count
    try:
        cr = driver.find_element(By.CSS_SELECTOR, 'rt-link[slot="criticsReviews"]').text.strip()
        movie['Critic_Review_Count'] = int(re.search(r'\d+', cr).group()) if cr else None
    except:
        movie['Critic_Review_Count'] = None

    # Audience Review Count
    try:
        ar = driver.find_element(By.CSS_SELECTOR, 'rt-link[slot="audienceReviews"]').text.strip()
        movie['Audience_Review_Count'] = int(re.search(r'\d+', ar).group()) if ar else None
    except:
        movie['Audience_Review_Count'] = None

    # Audience Score
    try:
        score = driver.find_element(By.CSS_SELECTOR, 'rt-text[slot="audienceScore"]').text.strip().replace('%', '')
        movie['Audience Score'] = int(score)
    except:
        movie['Audience Score'] = None

    driver.quit()
    return movie

# Read the CSV file with movie URLs
input_csv = "/content/drive/MyDrive/WebMining/rottentomatoes_500_movies_by_genre.csv"
df_urls = pd.read_csv(input_csv)

# Extract URLs from the column (assumes column name is 'url')
movie_urls = df_urls['url'].dropna().tolist()

# Placeholder for movie metadata
all_movies = []

# Loop through URLs and scrape
for url in tqdm(movie_urls, desc="Scraping Movies"):
    try:
        movie_data = scrape_movie_data_selenium(url)  # Use your existing function
        all_movies.append(movie_data)
        time.sleep(1)  # Optional polite delay
    except Exception as e:
        print(f"❌ Failed to scrape {url} -> {e}")
        continue

# Convert to DataFrame and save
df = pd.DataFrame(all_movies)
df.to_csv("rottentomatoes_movies.csv", index=False)
print("\n✅ Done! Data saved to rottentomatoes_movies.csv")

Scraping Movies: 100%|██████████| 500/500 [2:06:42<00:00, 15.20s/it]


✅ Done! Data saved to rottentomatoes_movies.csv


In [ ]:
from google.colab import files
files.download('rottentomatoes_movies.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>